## OECD Data Download

In [1]:
## save the installed packages in the google drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

!pip install --target=$nb_path pandasdmx
!pip install --target=$nb_path requests_cache
!pip install --target=$nb_path pandas_datareader
!pip install --target=$nb_path cif

Mounted at /content/drive


In [3]:
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import pandas_datareader.data as web
import json
import requests
from google.colab import drive
from tqdm import tqdm

path='drive/MyDrive/Capstone(Team10)/code'

In [4]:
## download MEI data for listed countries using OECD API
## long runtime: run only once for initial downloading

country_list=['USA']  ##'USA', 'KOR', 'JPN'

for country in tqdm(country_list):
     df_MEI=pd.read_csv('https://stats.oecd.org/SDMX-JSON/data/MEI/{}...M/all/OECD?contentType=csv'.format(country))
     df_MEI.to_csv(path+'/data/MEI_{}.csv'.format(country))

In [9]:
## make dataframe using downloaded data and see # of features and length
data=pd.read_csv(path+'/data/MEI_USA.csv')
len(data.Subject.unique()), min(data.TIME.unique()), max(data.TIME.unique())

<ipython-input-9-a86a671f33ca>:1: DtypeWarning: Columns (15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(path+'/data/MEI_USA.csv')


(314, '1919-01', '2023-06')

In [10]:
## creating metadata for OECD data
metadata=data[['SUBJECT','Subject','Measure','Frequency','Unit','Reference Period']].drop_duplicates().reset_index()
metadata['Level']=metadata['Subject'].apply(lambda x:x.split(' > ')[0])
metadata['Name']=metadata['Subject'].apply(lambda x:' '.join(x.split(' > ')))

## save metadata into folder and copy this file into "metatdata_filter.xlsx" to manually select relevant features.
## 753 features ==> 63 features
metadata.to_csv(path+'/data/metadata_US.csv')
metadata.head()

,index,SUBJECT,Subject,Measure,Frequency,Unit,Reference Period,Level,Name
0,0,LCEAPR01,Labour Compensation > Earnings > Private secto...,"Level, rate or national currency, s.a.",Monthly,US Dollar,NaN,Labour Compensation,Labour Compensation Earnings Private sector Ho...
1,713,LCEAPR01,Labour Compensation > Earnings > Private secto...,"Index 2015=100, s.a.",Monthly,Index,2015=100,Labour Compensation,Labour Compensation Earnings Private sector Ho...
2,1426,XTNTVA01,International Trade > Net trade > Value (goods...,"National currency, monthly level",Monthly,US Dollar,NaN,International Trade,International Trade Net trade Value (goods) Total
3,2246,XTNTVA01,International Trade > Net trade > Value (goods...,"National currency, monthly level, s.a.",Monthly,US Dollar,NaN,International Trade,International Trade Net trade Value (goods) Total
4,3066,XTNTVA01,International Trade > Net trade > Value (goods...,"US Dollars, monthly level, s.a.",Monthly,US Dollar,NaN,International Trade,International Trade Net trade Value (goods) Total


## Data Preparation

### 1) Manual Feature Selection

In [11]:
## please run this cell after you change your feature list on metadata_filter.xlsx file
## copy the filtered list of metadata and save it as "metadata_final.csv" file

df=pd.read_excel(path+'/data/metadata_filter.xlsx', sheet_name='list')
df.to_csv(path+'/data/metadata_final.csv')

### 2) Data Preprocessing

* Filtering the data with selected variables and Reshaping the DataFrame from long to wide format

* Updating Metadata with information about if the variable is the country of interest

* features & sources
 * features: 52 features from OECD + 11 features from other sources(FRED, Yahoo Finance)
 * target variables: 10 variables relevant to target label(OECD, NBER, FRED)


In [13]:
## importing stored data and metadata with feature list
country='USA'
data=pd.read_csv(path+'/data//MEI_{}.csv'.format(country))

df=pd.read_csv(path+'/data/metadata_final.csv')

filter_df=df[['SUBJECT','Measure']]

keys = list(filter_df.columns.values)
i1 = data.set_index(keys).index
i2 = filter_df.set_index(keys).index
data1=data[i1.isin(i2)]

## print the number of features
print('the number of existing features:', len(data1.SUBJECT.unique()))  ## 59 features for OECD, 4 features for other DB

## transform TIME to datetime index
data1['TIME']=pd.DatetimeIndex(data1['TIME'])

## transform dataframe from long to wide format
filter_data=pd.pivot_table(data1, values='Value', index='TIME', columns= 'SUBJECT')
filter_data.index.names = ['date']
filter_data.columns.name = None

ftlist=list(df['SUBJECT'].unique())
data_ftlist=list(filter_data.columns)
missing_ftlist=[x for x in ftlist if x not in data_ftlist][:(59-len(df))] ## except 11 manually downloaded features from other sources

df['missing_{}'.format(country.lower())]=df['SUBJECT'].apply(lambda x:1 if x in missing_ftlist else 0)

## uncomment to save the change in the files
df.to_csv(path+'/data/metadata_final.csv', index=False)

## check name of missing features in the dataset
missing_ftnames=[df[df.SUBJECT==x]['Name'].values[0] for x in missing_ftlist]
print('the number of missing features:', len(missing_ftnames))
missing_ftnames

<ipython-input-13-877932f08bb6>:3: DtypeWarning: Columns (15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(path+'/data//MEI_{}.csv'.format(country))


the number of existing features: 59
the number of missing features: 0


<ipython-input-13-877932f08bb6>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['TIME']=pd.DatetimeIndex(data1['TIME'])


[]

In [14]:
## import data from additional feature from other sources
other_data=pd.read_csv(path+'/data/other_data_{}.csv'.format(country))


## transform TIME to datetime index
other_data['Date']=pd.DatetimeIndex(other_data['Date'])
other_data=other_data.set_index('Date', drop=True)
other_data.head()

raw_data=filter_data.merge(other_data, left_index=True, right_index=True)
raw_data.to_csv(path+'/data/rawdata_{}.csv'.format(country))

In [15]:
raw_data.tail()

,BNBUCT02,BNEMTE02,BNODTE02,BNRMTE02,BSCICP02,BSCURT02,BSEMFT02,BSOBLV02,BSOITE02,BSPRTE02,...,BBBOASPREAD,SP500,WTI,W875RX1,VIXCLS,RECPROUSM156N,DXY,y_nber,y_oecd,y_agg
2023-02-01,NaN,NaN,NaN,NaN,-2.6,78.4,-1.488108,-10.0,-9.904271,-6.838155,...,1.55,3970.153442,76.83,14622.7,20.12,0.36,104.870003,0,0.0,0.0
2023-03-01,NaN,NaN,NaN,NaN,-3.7,77.8,-4.394159,-10.0,-11.664475,-4.339667,...,1.77,4109.312445,73.28,14664.3,21.64,0.32,102.510002,0,0.0,0.0
2023-04-01,NaN,NaN,NaN,NaN,-2.9,78.4,-0.707885,-11.0,-10.242475,-4.047127,...,1.73,4169.481401,79.45,14659.0,17.82,0.22,101.669998,0,0.0,0.0
2023-05-01,NaN,NaN,NaN,NaN,-3.1,78.4,2.680256,-13.0,-17.834084,1.060291,...,1.80,4179.825462,71.58,14702.7,17.64,0.62,104.320000,0,0.0,0.0
2023-06-01,NaN,NaN,NaN,NaN,-4.0,NaN,-5.300056,-9.0,-6.879287,-6.892674,...,1.69,4450.381312,68.11,NaN,14.00,NaN,102.910004,0,0.0,0.0


In [16]:
## show the available years(without na values) for each variables
round((len(raw_data)-raw_data.isna().sum())/12,2)

BNBUCT02         23.00
BNEMTE02         23.00
BNODTE02         23.00
BNRMTE02         23.00
BSCICP02         73.50
                 ...  
RECPROUSM156N    33.42
DXY              38.50
y_nber           89.50
y_oecd           73.75
y_agg            73.75
Length: 73, dtype: float64